# Collaborative filtering
- Patrick Schaper (534366)
- Daniel-Alexandru Bejan (474404)

In [ ]:
from IPython.core.display import HTML
from movie_display import movie_display
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity


## Dataset analysis

In this section of the notebook i am doing some analysis on the data to get to know the dataset better and get some more insights about the data and what we're dealing with.

In [ ]:
# Load into a dataframe
movies = pd.read_csv('./dataset/movies.csv')
links = pd.read_csv('./dataset/links.csv')
ratings = pd.read_csv('./dataset/ratings.csv')
tags = pd.read_csv('./dataset/tags.csv')
df = pd.read_json('./dataset/imdbdata.json', orient='columns')


This cell is to get some basic information about the data

In [ ]:
print(f"Number of ratings: {len(ratings)}")
print(f"Number of unique movieId's: {ratings['movieId'].nunique()}")
print(f"Number of unique users: {ratings['userId'].nunique()}")
print(f"Average number of ratings per user: {round(len(ratings)/ratings['userId'].nunique(), 2)}")
print(f"Average number of ratings per movie: {round(len(ratings)/ratings['movieId'].nunique(), 2)}")

In [ ]:
sns.countplot(x="rating", data=ratings)
plt.title("Distribution of movie ratings")
plt.show()

We are trying to find out in this section with is the global mean rating and the mean rating for user.

In [ ]:
print(f"Mean global rating: {round(ratings['rating'].mean(),2)}.")

mean_ratings = ratings.groupby('userId')['rating'].mean()
print(f"Mean rating per user: {round(mean_ratings.mean(),2)}.")

Let's check out the lowest rated movie, which for some reason is the "Santa with Muscles"

In [ ]:
mean_ratings = ratings.groupby('movieId')[['rating']].mean()
lowest_rated = mean_ratings['rating'].idxmin()
movies[movies['movieId'] == lowest_rated]

And the highest rated movie is "Lamerica"

In [ ]:
highest_rated = mean_ratings['rating'].idxmax()
movies[movies['movieId'] == highest_rated]

In [ ]:

movie_stats = ratings.groupby('movieId')[['rating']].agg(['count', 'mean'])
movie_stats.columns = movie_stats.columns.droplevel()

In [ ]:
# C = movie_stats['count'].mean()
# m = movie_stats['mean'].mean()

# def bayesian_avg(ratings):
#     bayesian_avg = (C*m+ratings.sum())/(C+ratings.count())
#     return bayesian_avg

# bayesian_avg_ratings = ratings.groupby('movieId')['rating'].agg(bayesian_avg).reset_index()
# bayesian_avg_ratings.columns = ['movieId', 'bayesian_avg']
# movie_stats = movie_stats.merge(bayesian_avg_ratings, on='movieId')

In [ ]:
# movie_stats = movie_stats.merge(movies[['movieId', 'title']])
# movie_stats.sort_values('bayesian_avg', ascending=False).head()

In [ ]:
# movie_stats.sort_values('bayesian_avg', ascending=True).head()

In [ ]:

# movies['genres'] = movies['genres'].apply(lambda x: x.split("|"))
# movies.head()

### Number of movie genres

In [ ]:
from collections import Counter

genre_frequency = Counter(g for genres in movies['genres'] for g in genres)

print(f"There are {len(genre_frequency)} genres.")

genre_frequency

Checking out the top 5 most common genres

In [ ]:
print("The 5 most common genres: \n", genre_frequency.most_common(5))

## User-item matrix
After analizing the dataset a little we are constructing the User-Item matrix and we start calculating the similarities.

In [ ]:
# Create a pivot table with userId as the index, movieId as the columns, and rating as the values
user_item_matrix = ratings.pivot_table(index='userId', columns='movieId', values='rating')

In [ ]:
user_item_matrix

# Similarity metrics

This section is about the similarity metrics that we are going to use to calculate the similarities between the users and the items.

In [ ]:
def calculate_similarity(df, similarity='none'):
    """
    Prepare the datafram for the cosine similarity
    Parameters
    ----------
    df : Pandas Dataframe
    similarity : string
    Returns
    -------
     dataframe : Pandas dataframe
    """
    if similarity == 'pearson':
        # centered zero matrix
        # From slides: Subtract the mean (average) user rating from each user’s rating
        # (substracts the ROW mean)
        # 1. Subtract the average of a user from its ratings
        # 2. The negative values represent negative ratings, positive values represent positive ratings
        # 3. The value 0 is now the average rating for a user
        similarity_matrix = df.subtract(df.mean(axis=1), axis=0)   
    if similarity == 'adjusted':
        # Adjusted cosine similarity
        # Per slides: Subtract the average item rating from each user rating for a given item (when
        # we calculate the difference between users)
        # (substracts the COLUMN mean)
        similarity_matrix = (df - df.mean())
    # From slides: 
    # Problem: low ratings and high ratings will have the same angle when
    # calculating the cosine similarity
    # ▪ Difference between A and B and A and C is small, whereas A and C are
    # almost opposite users
    # ▪ Solution: center around 0
    # Therefore we have to center everything around 0 filling the NaN values with 0. as follows
    similarity_matrix = df.fillna(0)
    # Calculating the cosine similarity
    similarity = cosine_similarity(similarity_matrix)
    # set the columns and index of the initial dataframe otherwise it would messup the indexes later
    return pd.DataFrame(similarity,index=df.index, columns=df.index)

#### Calculate similarity matrixes

Here we test our similarity matrix calculation functions and we are going to use them later on in the notebook.

In [ ]:
cosine_similarity_matrix = calculate_similarity(user_item_matrix)

pearson_similarity_matrix = calculate_similarity(user_item_matrix, 'pearson')

ajusted_cosine_similarity_matrix = calculate_similarity(user_item_matrix, 'adjusted')

# Example for each similarity

### Cosine Similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
# cosine similarity
cosine_similarity_matrix = calculate_similarity(user_item_matrix)
# calc pearson similarity
pearson_similarity_matrix = calculate_similarity(user_item_matrix, 'pearson')
# calculate adjusted cosine similarity
adjusted_cosine_similarity_matrix = calculate_similarity(user_item_matrix, 'adjusted')

In [ ]:
# sns.heatmap(cosine_similarity_matrix, cmap='viridis')
# sns.heatmap(pearson_similarity_matrix, cmap='viridis')
# sns.heatmap(ajusted_cosine_similarity_matrix, cmap='viridis')

## User-user recommendations

#### Converting the user-item matrix to user-user matrix

In [ ]:
cosine_similarity_matrix

The cell below is just a testing that we have used to check the user-item matrix.

In [ ]:
user_id = 1
user_row = cosine_similarity_matrix.loc[1]

updated_df = user_row.drop(1)
updated_df.head()
updated_df.sort_values(ascending=False)

The method below is used to get the top 10 similar users for a given user_id and user_item matrix as well as the similarity strategy.

In [ ]:
def get_similar_users(user_id, user_item_matrix, similarity_strategy='none'):
    """"Create a dict with the most similar users and ranks them based on similarity """
    # get the similarity matrix based similarity strategy
    similarity_matrix = calculate_similarity(user_item_matrix, similarity_strategy)
    # drop the user itself
    similar_users = (similarity_matrix.loc[user_id]).drop(1)
    # sorting the value descending in order to get the most similar users first
    return similar_users.sort_values(ascending=False).head(10)

In [ ]:
# # testing the method above
# # user_id= 2
# similar_users = get_similar_users(2)

# similar_users

# # print(similar_users.index.to_list)


# # user_item_matrix.loc[similar_users.index]

# # for user_id, similarity in pd.DataFrame(similar_users).iterrows():
# #     print(user_id)
#     # simlar_user = user_item_matrix.iloc[user_id].dropna()
#     # print(simlar_user)

# # users_movies = pd.DataFrame(user_item_matrix.iloc[lambda x: x % 2 == 0])
# # print(type(users_movies.columns)) 
# # droplist = [column for column in users_movies if 1 >= column <= 5]
# # print(droplist)
# # dropped_columns = users_movies.drop(droplist,inplace=True)

# # dropped_columns.head()

In [ ]:
def get_recommendation_list(user_id, number_of_recommendations, similarity_strategy, debug=False):
    # get the similar users with the selected similarity strategy as pandas dataframe
    similar_user_data = get_similar_users(user_id, user_item_matrix, similarity_strategy)

    # print(similar_users)
    # get pandas dataframe with each users and their movies
    rated_movies_by_user = user_item_matrix.loc[user_id].dropna()
    # users_movies = pd.DataFrame(user_item_matrix.iloc[2])
    recommendations = []
    for user_id, similarity in similar_user_data.items():
        # select the user from the user_item matrix and drop the null values
        similar_user_movies = user_item_matrix.loc[user_id].dropna()
        # print('current user',similar_user_movies.tolist())
        for movie_id, rating in similar_user_movies.items():
            if movie_id not in rated_movies_by_user:
                # print('user_id', user_id, 'movie', movie_id, 'similarity',similarity, 'rating', rating)
                recommendations.append((movie_id, rating * similarity))
    # sort values descending
    # recommendations = initial_recommendation.sort_values(ascending=False).head(number_of_recommendations)
    # # return the movies id only
    # print(recommendations[:30])
    recommendations.sort(key=lambda tup: tup[1], reverse=True)
    # print(recommendations[:10])
    if (debug):
        print(len(recommendations))
        print(recommendations[:10])
    movies = []
    # put the movie ids into a list to be prepared for display in the notebook widgets
    for movie_id, _ in recommendations:
        movies.append(movie_id)
    
    return movies[:number_of_recommendations]

Let's see what we are getting for user_id, and with the default cosine similarity.
Observing the results we can see the the method is working as expected. Giving us the top 5 movies with their similarities score.

In [ ]:
get_recommendation_list(2, 5, 'none', debug=True)

## Item-item recommendations

1. Select a user
2. Select the movies that the user has rated more than 3.5 stars
3. Based on the user 

Give N (configurable) recommendations for a given user U (configurable) based on the
movies the user U rated with at least 3.5 stars. Explain your implementation and the
strategy that you use for selecting the final recommendations.
Which means that we have to give the user N number of recommendations for a certain user(selectable) from the movies that he has rated with at least 3.5 stars.

In the UI we would have to:
1. N number of recommendations
2. U which user id
3. Select certain movie which is rated >= 3.5 stars

How to do the recommendations:
1. find similar items
2. Candidate selection (items you might recommend)
3. Score recommendation candidates
4. Filter candidates (top_n) recommendation

In [ ]:
# Create item_user matrix
# The difference between the user_item matrix is that the movie id will be the index and the columns would be the user id
# Which means that when we calculated the similarity matrix we would calculate the mean and using the ratings from the users 
# Per slides: Both(pearson and adjusted cosine similarity) are applicable for user-user and item-item recommendations
# Simply switch around users and items in the formulas!
item_user_matrix = ratings.pivot_table(index='movieId', columns='userId', values='rating')

item_user_matrix.head(10)

Calculating the similarity matrixes

once again but now using the item_user_matrix

In [ ]:
# cosine similarity
item_item_cosine_similarity_matrix = calculate_similarity(item_user_matrix)
# calc pearson similarity
item_item_pearson_similarity_matrix = calculate_similarity(item_user_matrix, 'pearson')
# calculate adjusted cosine similarity
item_item_adjusted_cosine_similarity_matrix = calculate_similarity(item_user_matrix, 'adjusted')

In [ ]:
item_item_pearson_similarity_matrix.head(10)

For each similar item that has not been seen by the user, calculate the expected rating
Using the similarity matrixes 

In order to predict the rating we have to know the following and use the matrixes
We have to know the movie id to which we want to predict and the user id as well

Following we need the similarity matrix to find similar items and the item_user matrix
Prediction formula used is:

R(m, u) = {∑ ⱼ S(m, j)R(j, u)}/ ∑ ⱼ S(m, j)
R(m, u): the rating for movie m by user u
S(m, j): the similarity between movie m and movie j
j ∈ J where J is the set of the similar movies to movie m

source: https://medium.com/@Sumeet_Agrawal/item-based-collaborative-filtering-4e64f65ae6ea

In [ ]:
def rating_prediction(movie_id, user_id, user_item_matrix, similarity_matrix, debug=False):
    # rating from the user 
    # contains movie_id and the rating
    user_rating = user_item_matrix.loc[:, user_id].sort_values(ascending=False).dropna()
    # print('rating from user',user_rating)
    # the weights for our calculation
    # We look at the similarities between movie_id and the other movies 
    # and we sort and descending and pick 10 most similar items
    similarity_items_by_id = similarity_matrix.loc[movie_id][user_rating.index]
    # get the items that have a similarity greater than 0
    similarity_items_by_id = similarity_items_by_id[similarity_items_by_id > 0]
    # print('similar items by id',similarity_items_by_id)
    # as the weights sum to zero, can't be normalized
    if similarity_items_by_id.sum() == 0:
        # we have to calculate the mean of the user ratings and return it 
        return round(user_rating.mean(), 1)

    weighted_average = np.average(user_rating.loc[similarity_items_by_id.index], weights = similarity_items_by_id)
    # https://towardsdatascience.com/3-ways-to-compute-a-weighted-average-in-python-4e066de7a719
    predicted_rating = round(weighted_average, 1)
    if (debug):
        print('predicted', predicted_rating)
        print('user ratings', user_rating.loc[similarity_items_by_id.index])
        print('similar items by id',similarity_items_by_id)
    return predicted_rating

Once again testing the rating prediction

In [ ]:
# movie_id 1208 user_id 571 expected  3.5
rating_prediction(1208, 571, item_user_matrix, item_item_pearson_similarity_matrix, debug=True)
rating_prediction(2, 4, user_item_matrix, item_item_pearson_similarity_matrix, debug=True)

In [ ]:
def recommend_movies_by_item(user_id, number_of_recommendations, item_user_matrix, user_item_matrix, similarity_matrix, debug=False):
   """
    Recommend Movies for a user, used for Item-Item
    Parameters
    ----------
    user_id : integer
    number_of_recommendations : integer
    item_user_matrix : Pandas dataframe
    user_item_matrix : Pandas dataframe
    similarity_matrix : Pandas dataframe
    debug : boolean False as default
    Returns
    -------
     recommendation_list : [(movie_id, predicted_rating)]
    """
   # 1. Take the k highest rated items of a user, for this we need the ones with more than 3.5
   highest_rated_items = user_item_matrix.loc[user_id].loc[lambda rating : rating >= 3.5].sort_values(ascending=False)
   # Filter ones rated with more than 3.5
   # highest_rated_items.loc[lambda rating : rating >= 1]
   # 2. Find j similar items to those k highest rated items
   similar_items_to_highest_items = similarity_matrix.loc[highest_rated_items.index]
   if debug:
      print('highest rated items index, which is the id of the movie',highest_rated_items.index)
      print('highest rated items', highest_rated_items)
      print('similar items', similar_items_to_highest_items)
   # 3. For each similar item that has not been seen by the user, calculate the expected rating
   # checking if the value is NaN by checking if it's equals to itself https://stackoverflow.com/a/944712
   unrated_items_by_user = user_item_matrix.loc[user_id].loc[lambda rating : rating != rating]
   if debug:
      print('unrated movies', unrated_items_by_user)
   recommendation_list = []
   for movie_id, _ in unrated_items_by_user.items():
      # print('movie_id', movie_id)
      predicted_rating = rating_prediction(movie_id, user_id, user_item_matrix, similarity_matrix)
      recommendation_list.append((movie_id, predicted_rating))
   # 4. Select the top n movies with the highest rating
   recommendation_list.sort(key=lambda tup: tup[1], reverse=True)
   return recommendation_list[:number_of_recommendations]

Let's test the method above to see if it works by using the user_id 5 with the pearson similarity matrix

In [ ]:
recommend_movies_by_item(5, 3, item_user_matrix, user_item_matrix, item_item_pearson_similarity_matrix, debug=True)

# Validation
Possible validation methods:
- RMSE
- Hit-rate

In [ ]:
def calculate_hit_rate(test_dataframe, train_dataframe,similarity_matrix, debug=False):
    # User-User
    # Initialize the hit rate
    hit_rate = 0
    # Iterate over the users in the test set
    # print(test_df.index)
    # print(test_df.columns)
    # print('train', train_df)
    for movie_id, row in test_dataframe.iterrows():
        # print(type(row))
        for user_id, rating in row.items():
            # Select the row in the similarity matrix corresponding to the user
            user_row = similarity_matrix.loc[user_id]
            # Select the top N most similar users
            N = 10
            similar_users = user_row.sort_values(ascending=False).head(N)
            # print(similar_users)
            # Extract the user ids of the similar users
            similar_user_ids = similar_users.index
            # print(similar_user_ids)
            # Select the rows of the user-item matrix corresponding to the similar users
            print(similar_user_ids)
            similar_user_rows = train_dataframe[train_dataframe.isin(similar_user_ids)]

            # similar_user_rows = train_df.loc[similar_user_ids]
            # print(similar_user_rows)
            # Check if the recommended item is present in the test set for the user
            # print('hit rate', movie_id, 'hit2 ', similar_user_rows.columns)
            if movie_id in similar_user_rows.columns:
                hit_rate += 1
    # Calculate the hit rate
    hit_rate /= test_dataframe.shape[0]
    return hit_rate
    # Calculate the hit rate
    # hit_rate /= test_df.shape[0]
    # print(f' User-user Hit rate: {hit_rate:.2f}')

In [ ]:
def calculate_hit_rate_v2(user_item_matrix, similarity_matrix, debug=False):
    # User-User
    # 1. For each user, leave one high rating out (store this rating in a test set)
    hit_rate = 0
    N = 10
    user_number = len(user_item_matrix)
    for user_id, values in user_item_matrix.iterrows():
        # print('values', values)
        highest_rating = values.sort_values(ascending=False).index[0]
        # 2. Recommend n movies
        user_recommendations = get_user_reccommendations(user_id, 10, similarity_matrix)
        # 3. If the "left-out"-movie is part of your recommendation, you've got a hit!
        # print('user_recommendations', user_recommendations)
        # 3. If the "left-out"-movie is part of your recommendation, you've got a hit!
        if debug:
            print('checking',movie_id , user_recommendations)
        for movie_id ,rating in user_recommendations:
            if highest_rating == rating:
                hit_rate += 1
    return hit_rate / float(user_number)
# print('User-user Hit rate:', hit_rate / float(user_number))

In [ ]:
# Split the data into a training set and a test set
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(user_item_matrix, test_size=0.2)


In [ ]:
# User-User Hit-rate validation
# Split the data into a training set and a test set
# train_df, test_df = train_test_split(user_item_df, test_size=0.2)

# RMSE

1. Remove some test data from the dataset (remove those ratings from the training set)
2. Predict ratings for the missing items
3. Compare to the real values in the test data set

In [ ]:
def calculate_rmse(test_dataframe, item_user_matrix, similarity_matrix):
    # Initialize the list of squared differences
    squared_differences = []
    # Iterate over the user-item pairs in the test set
    for user_id, row in test_df.iterrows():
        for movie_id in row.index:
            # Check if the row value is nana if so continue
            if (row[movie_id] != row[movie_id]):
                continue
            # Calculate the predicted rating
            predicted_rating = rating_prediction(movie_id, user_id, item_user_matrix, similarity_matrix)
            # print('movie_id', movie_id, 'user_id', user_id, 'expected ', row[movie_id],'predicted',predicted_rating)
            # Calculate the squared difference between the predicted and actual rating
            squared_differences.append((predicted_rating - row[movie_id]) ** 2)

    # Calculate the MSE as the average of the squared differences
    mse = np.mean(squared_differences)
    # Calculate the RMSE as the square root of the MSE
    return np.sqrt(mse)

In [ ]:
# Split the data into a training set and a test set
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(user_item_matrix, test_size=0.1)

In [ ]:
# Split the data into a training set and a test set
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(user_item_matrix, test_size=0.1)
test_df

In [ ]:
# on average this runs in 1.5 minutes
rmse_item_item_cosine = calculate_rmse(test_df, item_user_matrix, item_item_cosine_similarity_matrix)
rmse_item_item_pearson = calculate_rmse(test_df, item_user_matrix, item_item_pearson_similarity_matrix)
rmse_item_item_adjusted = calculate_rmse(test_df, item_user_matrix, item_item_adjusted_cosine_similarity_matrix)

print(f'RMSE Cosine: {rmse_item_item_cosine:.2f}')
print(f'RMSE Pearson: {rmse_item_item_pearson:.2f}')
print(f'RMSE Adjusted: {rmse_item_item_adjusted:.2f}')

# Widgets implementation 

In [ ]:
def get_indexes(imdb_id=[]):
    movies = []
    for x in range(len(imdb_id)):
        movieIndex = df.loc[df['imdbId'] == imdb_id[x]].index[0]
        movies.append(df.iloc[movieIndex])
    
    return movies

def displayRecommendations(recommendations=[]):
    movies = []
    for _, row in links.iterrows():
        if row['movieId'] in recommendations:
            movies.append(int(row['imdbId']))
    
    return get_indexes(movies)

This is just a test for the movies display method

In [ ]:
# display(displayRecommendations([589, 1196]))

recommendations = displayRecommendations([337, 318, 21])
print(type(recommendations)) 
HTML(movie_display.show(recommendations))

In [ ]:
def user_user_recommendations(selected_user, recommendations, similarity_strategy):
    if similarity_strategy == 0:
        recommendation_list = get_recommendation_list(selected_user, recommendations, similarity_strategy='none')
    if similarity_strategy == 1:
        recommendation_list = get_recommendation_list(selected_user, recommendations, similarity_strategy='pearson')
    if similarity_strategy == 2:
        recommendation_list = get_recommendation_list(selected_user, recommendations, similarity_strategy='adjusted')

    return recommendation_list

In [ ]:
user_user_recommendations(2,3,0)

In [ ]:
def item_item_recommendations(selected_user, recommendations, similarity_strategy):
    if similarity_strategy == 0:
        recommendation_list = get_recommendation_list(selected_user, recommendations, similarity_strategy='none')
    if similarity_strategy == 1:
        recommendation_list = get_recommendation_list(selected_user, recommendations, similarity_strategy='pearson')
    if similarity_strategy == 2:
        recommendation_list = get_recommendation_list(selected_user, recommendations, similarity_strategy='adjusted')

    return recommendation_list

In [ ]:
item_item_recommendations(2, 3, 2)

In [ ]:
from IPython.core.display_functions import clear_output
import ipywidgets as widgets

selected_user = widgets.Dropdown(
    options=list(x+1 for x in range(ratings['userId'].nunique())),
    description='Select a user:\n ',
    disabled=False,
    layout={'width': 'max-content'}
)
recommendations = widgets.IntText(
    min=0,
    value=3,
    description='Number of recommendations:\n ',
    disabled=False,
)
recommendation_method = widgets.RadioButtons(
    options=[('User-User',0), ('Item-Item',1)],
    description='Recommendation:',
    disabled=False
)
similarity_strategy = widgets.RadioButtons(
    options=[('Cosine',0), ('Pearson',1), ('Adjusted cosine',2)],
    description='Similarity metrics:',
    disabled=False
)
button = widgets.Button(
    description='Recommendation',
    disabled=False,
)

def execute_function(_):
    with out:
          clear_output()
          recommendation_list = []
          print(recommendation_method)
          if recommendation_method.value == 0:
            recommendation_list = user_user_recommendations(selected_user.value, recommendations.value, similarity_strategy.value)
          if recommendation_method.value == 1:
            recommendation_list = item_item_recommendations(selected_user.value, recommendations.value, similarity_strategy.value)
          print(f'Selected User: {selected_user.value}')
            
          print('Recommendation(s):')
          print(f'Got movies with the following ids as recommendations: {recommendation_list}')
          display(HTML(movie_display.show(displayRecommendations(recommendation_list))))
            
button.on_click(execute_function)
out = widgets.Output()

box = widgets.VBox([recommendations, selected_user, recommendation_method, similarity_strategy, button, out])
box